# Hybrid-RAG

# All Required Libearies

#### now 1st we will install all the required libraries

In [ ]:
!pip -q install langchain huggingface_hub pinecone pinecone_client tiktoken
!pip -q install -U unstructured pdfminer.six pymupdf sentence_transformers
!pip -q install pytesseract pdf2image Pillow
!apt-get install -y tesseract-ocr libtesseract-dev poppler-utils
!pip -q install pytesseract langchain_community
!pip -q install chromadb
!pip -q install langchain-google-genai
!pip -q install google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.

### Before starting the project 1st we will ensure that the drive is mounted or not and also we are using "cuda" as a device for the faster outcome

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## importing the required Libraries

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import Pinecone
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#### Now load the document that you want for the Context

In [ ]:
path = "/content/drive/MyDrive/Internship projects/Sample Set/docs"
document = PyPDFDirectoryLoader(path)
data = document.load()

In [ ]:
len(data)

64

#### Text Splitting

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 512 , chunk_overlap = 150, length_function = len , is_separator_regex = False)
chunks = [splitter.split_text(doc.page_content) for doc in data]
flat_chunks = [chunk for sublist in chunks for chunk in sublist]


In [ ]:
splitter1 = RecursiveCharacterTextSplitter(chunk_size=800,
                                          chunk_overlap=100)
chunk = splitter1.split_documents(data)

In [ ]:
import pandas as pd

df = pd.DataFrame(flat_chunks , columns = ['vectors'])
df

,vectors
0,Submitte d to the 3rd International Conference...
1,msholia@bvmengineering.ac.in \nBirla Vishvaka...
2,"customer service using LangChain, a custom LLM..."
3,"open -source methodologies, web scraping, fine..."
4,"collection via web scraping, the role of embed..."
...,...
715,"Clement Delangue, Anthony Moi, Pierric Cistac,..."
716,"Norouzi, Wolfgang Macherey, Maxim Krikun, Yuan..."
717,{Transformer-Based }Generative Models. In 16th...
718,"[62] Susan Zhang, Stephen Roller, Naman Goyal,..."


#### Loading the HuggingFace Embedding Model for creating an Embeddings of chunks that we created Earlier

#### This Well known Embedding Model:"BGE Model" Which is availbale on HuggingFace

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {'normalize_embeddings' : True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name = model_name ,
    encode_kwargs= model_kwargs,
    model_kwargs = {'device':'auto'},
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Pinecone VectorStore setup (Implemented using Pinecone API-Key)

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
import os
api_key = os.environ.get("PINECONE_API_KEY")
index_name = "apurv"
pc = Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws', region='us-east-1'),
    )

index = pc.Index(index_name)
upsert_vectors = [
    (str(i), embeddings.embed_documents([chunk])[0], {"metadata_key": "metadata_value"})
    for i, chunk in enumerate(df['vectors'])
]
index.upsert(upsert_vectors)

{'upserted_count': 720}

### Now setting up the another retriever "Chroma"

In [ ]:
vectorstore = Chroma.from_documents(chunk, embeddings)

#### In HyBrid-RAG we use Ensamble Retreiver which requires Two diffrent retreivers and then it provides output based on the weights we assign to both retreivers

##### so here is the Setup of Pinecone_vecotrstore retreiver and Chromadb_vectorStore retreiver

In [ ]:
from langchain.retrievers import EnsembleRetriever
import pinecone
import os
from langchain.vectorstores import Pinecone as LangChainPinecone
from pinecone import Pinecone, ServerlessSpec
# Initialize Pinecone client
import os
api_key = os.environ.get("PINECONE_API_KEY")
index_name = "apurv"
pc = Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws', region='us-east-1'),
    )


index = pc.Index(index_name)

retriever1 = vectorstore.as_retriever(search_kwargs={"k": 3})
retriever2 = LangChainPinecone(

    index=index,
    embedding=embeddings.embed_query,
    text_key="vectors"
).as_retriever(search_kwargs={"k": 3})


ensambel_reteriever = EnsembleRetriever(
    retrievers = [retriever1 , retriever2],
    weight = [0.6,0.4]

)

<ipython-input-14-197dac123089>:23: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  retriever2 = LangChainPinecone(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


# Gemini LLM setup

In [ ]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key = os.environ["GOOGLE_API_KEY"])

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI( model = 'gemini-1.5-flash' , temperature= 0.7)

# Prompt Template Designing

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """
You are a knowledgeable assistant skilled in providing thorough and accurate responses. Use the provided context to answer the user's question comprehensively. Ensure that your answer covers all relevant aspects and is presented in a clear and organized manner and also try to give a full complete response .
Context:
{context}

Question:
{query}

Answer (detailed, with examples if relevant):
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , template),
        ("human" , '{input}')
    ]
)

output_parser = StrOutputParser()

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
import logging
logging.getLogger("langchain_community.vectorstores.pinecone").setLevel(logging.ERROR)

# Creating A QA Chain using Langchain and combining Prompt , Context and then it provided to LLM

In [ ]:
chain = RunnableMap({
    "context": lambda x: ensambel_reteriever.get_relevant_documents(x["query"]),
    "query": lambda x: x["query"],
    "input": lambda x: x["query"]
}) | prompt | llm | output_parser

# Prepare the input data
input_data = {
    "query": "What are the advantages and disadvantages of vLLM?"
}

# Invoke the chain
result = chain.invoke(input_data)
print(result)

## Advantages of vLLM:

* **Improved Throughput:** vLLM significantly boosts the throughput of popular LLMs, achieving a 2-4x improvement compared to state-of-the-art systems like FasterTransformer and Orca. This enhanced performance is particularly noticeable with longer sequences, larger models, and more complex decoding algorithms.
* **Near-Zero Waste in KV Cache Memory:** vLLM effectively minimizes waste in the KV cache memory, making optimal use of available resources. This efficient utilization of memory contributes to its high performance.
* **Flexible Sharing of KV Cache:** vLLM enables flexible sharing of KV cache both within and across requests, further reducing memory usage. This sharing mechanism allows for more efficient utilization of memory resources.
* **LLM-Specific Optimizations:** vLLM incorporates LLM-specific optimizations to its virtual memory and paging techniques. These optimizations, such as the "all-or-nothing" swap-out policy and the recomputation method for 

In [ ]:
input_data = {
    "query":"can you explain AutoDAn"
}
result = chain.invoke(input_data)
result

'AutoDAN is a semi-automated method for generating test cases to elicit specific harmful behaviors from large language models (LLMs). It operates through a two-step process:\n\n1. **Initialization:** AutoDAN starts by using handcrafted "jailbreak" prompts as initial test cases. These prompts are designed to exploit vulnerabilities in the LLM\'s safety mechanisms and encourage it to produce undesirable outputs.\n\n2. **Evolution:** The initial test cases are then evolved using a hierarchical genetic algorithm. This algorithm iteratively modifies the prompts based on the LLM\'s responses, aiming to refine them and generate more effective test cases that trigger the desired harmful behaviors.\n\n**Here\'s a breakdown of how AutoDAN works:**\n\n* **Handcrafted Jailbreak Prompts:** These are carefully designed prompts that aim to bypass the LLM\'s safety filters and prompt it to engage in harmful behaviors. For example, a jailbreak prompt might try to trick the LLM into generating biased or

In [ ]:
chain.invoke({"query":"can you explain What is GBDA from the given context"})

'The provided context does not mention "GBDA". Therefore, I cannot explain what GBDA is based on the given information. \n\nIt\'s possible that "GBDA" is a term or acronym used in a different context or document. If you can provide more information or context, I can try to help you understand its meaning. \n'